### Read the feature engineered data

In [1]:
import pandas as pd
import os
import random
filename = r'C:/Users/endwy/Documents/Columbia MSBA/Spring 2019/E4650 - Business Analytics/Project - Kickstarter Analysis/raw_data/df_kickstarterFE.csv'

p = 1  # x% of the lines
# keep the header, then take only x% of lines
# if random from [0,1] interval is greater than p the row will be skipped
df = pd.read_csv(filename, header=0,  skiprows=lambda i: i>0 and random.random() > p)
len(df)
#df=df[:1000]
#len(df)
df.head()

,Unnamed: 0,Launch_deadline_days,Cat_Name,country,staff_pick,state,backers_count,blurb,goal,launched_at,usd_pledged,deadline,name
0,0,30,0,0,0,0,108,"In the year 2025, a young man's superintellige...",4000.0,0,5265.820000,0,A Date in 2025 - A sci-fi comedy short film
1,1,44,1,0,1,0,37,Replacement Place needs your support to combat...,5000.0,1,5225.000000,1,Replacement Place | New York Live Arts
2,2,14,2,0,0,0,27,Pumpkin Spice Love - A Pumpkin Spice 4 Life 30...,250.0,0,362.000000,2,Pumpkin Spice Love!
3,3,30,3,0,0,0,184,Single mama and her 4-year old sail with all-f...,15000.0,2,15678.000000,3,Pirate Mama
4,4,30,4,1,0,0,169,SparKit wishes to produce kit-set Wimshurst Ma...,2500.0,3,7978.645365,4,SparKit - Miniature Electrostatic Generator


In [2]:
df1 = df.drop(['Unnamed: 0','blurb','name','backers_count'],axis=1)

In [3]:
import numpy as np
df['usd_pledged'] = np.log(1+df['usd_pledged'])
df['goal'] = np.log(1+df['goal'])

## Splitting data into train and test

In [4]:
from sklearn.model_selection import train_test_split
train, test = train_test_split(df1, test_size = 0.3)
X_train = train.drop(['state'],axis=1)
y_train = train['state']
X_test = test.drop(['state'],axis=1)
y_test = test['state']

## XgBoost Model

In [5]:
#!pip install xgboost
import xgboost as xgb
from sklearn.model_selection import KFold, cross_val_score, train_test_split
from sklearn.preprocessing import LabelEncoder
import xgboost as xgb
from sklearn.feature_selection import SelectFromModel # Dimensionality reduction
from sklearn.model_selection import RandomizedSearchCV # Model evaluation
from sklearn.metrics import mean_absolute_error, mean_squared_error
from sklearn.ensemble import RandomForestRegressor

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\ensemble\weight_boosting.py:29: DeprecationWarning: numpy.core.umath_tests is an internal NumPy module and should not be imported. It will be removed in a future NumPy release.
  from numpy.core.umath_tests import inner1d


In [21]:
### XGBoost 

### prepare xgboost parameters
xgb_params = {
    'objective': 'binary:logistic' ,
    'max_depth':5,
    'gamma':0.1,
    'min_child_weight':5,
    'colsample_bytree':0.05
}

dtrain = xgb.DMatrix(X_train, y_train)

dtest = xgb.DMatrix(X_test)

num_boost_rounds = 200
model = xgb.train(dict(xgb_params, silent=1), dtrain, num_boost_round=num_boost_rounds)

xgb_pred1 = model.predict(dtest)


print(f'The mean absolute error using the original training data set is {mean_absolute_error(xgb_pred1, y_test)}')
print(f'The mean squared error using the original training data set is {mean_squared_error(xgb_pred1, y_test)}')

C:\ProgramData\Anaconda3\lib\site-packages\xgboost\core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \
C:\ProgramData\Anaconda3\lib\site-packages\xgboost\core.py:588: FutureWarning: Series.base is deprecated and will be removed in a future version
  data.base is not None and isinstance(data, np.ndarray) \


The mean absolute error using the original training data set is 0.08445766564812618
The mean squared error using the original training data set is 0.025331021082635366


In [8]:
xgb_pred1

array([1.0000000e+00, 1.8101639e-09, 1.0000000e+00, ..., 7.6351876e-05,
       1.0000000e+00, 1.2987060e-04], dtype=float32)

In [9]:
y_test.head()

79479     1
26934     0
576749    1
385889    1
140926    1
Name: state, dtype: int64

In [10]:
def get_classification(predictions,threshold):
    classes = np.zeros_like(xgb_pred1)
    for i in range(len(classes)):
        if predictions[i] > threshold:
            classes[i] = 1
    return classes

In [11]:
import numpy as np
from sklearn.metrics import confusion_matrix
confusion_matrix(y_test,get_classification(xgb_pred1 ,0.5))
tn, fp, fn, tp = confusion_matrix(y_test,get_classification(xgb_pred1,0.5)).ravel()
tpr = tp/(tp+fn)
fpr = fp/(fp+tn)


In [12]:
print(tpr, fpr)

0.9991578760748161 2.828694274722788e-05
